In [14]:
import pandas as pd
import numpy as np
import pymarc
import re
import csv


In [15]:
path= 'data/hertz.mrc'
out_file_path = 'data/csv_files/hertz.csv'

COLUMNS = ['id', 'signature', 'inventory_nr']

In [26]:
with open(path, 'rb') as fh:
    reader = pymarc.MARCReader(fh, to_unicode=True, force_utf8=True, utf8_handling='ignore')

    with open(out_file_path, 'w', encoding='utf8') as out:

         #Dictwriter for better performance, write header

            writer = csv.DictWriter(out, fieldnames=COLUMNS)
            writer.writeheader()

            rows = []

            for record in reader:
                
                id = record['001']
                signature = record ['LOD' ]['3'] if record ['LOD' ] != None else None
                inventory_nr = record ['LOD' ]['6'] if record ['LOD' ] != None else None

                rows += [{
                    'id': id.value(), 
                    'signature': signature, 
                    'inventory_nr': inventory_nr
                }]

                #print(id.value())
                #print(record)


            writer.writerows(rows)


            

In [50]:
import requests
from bs4 import BeautifulSoup

requests.packages.urllib3.disable_warnings()


# Make a request to the website and parse the HTML
url = 'https://dlib.biblhertz.it/service/aufstellung'
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')

# create an empty dictionary to store the results
result_dict = {}


level1_elements = soup.find_all(class_="level1")


# find the top-level list of categories
for element in level1_elements:
    link_text = element.find('a').text.strip()
   
    # create a dictionary to store the text and subfields of each element
    subfields_dict = {}

    # find all the child elements of the current element with class "level2"

    #THE LEVEL 2 TAGS ARE NOT ALWAYS CORRECT!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    level2_ul = element.find('ul', {'class': 'tree'})
     
    if level2_ul:
        level2_li = level2_ul.find_all('li')

        for li2 in level2_li:



Aa   Allgemeine Nachschlagewerke
Ab   Kunstwissenschaft
Ad   Ur- und Frühgeschichte
50   Enzyklopädien und Sachlexika
100   Bibliographien
200   Gelehrtenlexika und Adreßbücher
300   Künstlerlexika
500   Sonstiges
Ae   Archäologie
Af   Historische Hilfswissenschaften
Ag   Buch-, Bibliotheks- und Archivwesen
Ah   Geographie, Geologie, Astronomie
Ai   Politische Geschichte
50   Enzyklopädien und Sachlexika
Ak   Kirchengeschichte
Al   Kultur-, Sozial- und Sittengeschichte, Volks- und Völkerkunde
50   Enzyklopädien und Sachlexika
100   Bibliographien
200   Gelehrtenlexika und Adreßbücher
500   Sonstiges
Am   Musik-, Tanz- und Theatergeschichte
An   Literaturgeschichte
Aq   Mythologie
Ar   Übrige Geistesgeschichte (ohne Theologie)
At   Theologie und Frömmigkeit (Weltreligionen)
Ax   Naturwissenschaft und Technik
Az   Sprachwörterbücher
20   Polyglott-Ausgaben, allgemeiner Wortschatz
24   idem, kunstgeschichtlicher Wortschatz
28   idem, sonstiger spezieller Wortschatz
40   Hebräisch
50   Alt

In [39]:
result

{'A': {'topic': "Â\xa0 Â\xa0 \nA Â\xa0 HandbÃ¼cherei\n\n Â\xa0 Â\xa0 \nAa Â\xa0 Allgemeine Nachschlagewerke\n\n Â\xa0 Â\xa0 \n Â\xa0 Lexika\n\n Â\xa0 Â\xa0 \n40 Â\xa0 EnzyklopÃ¤dien und Sachlexika\n\n\n\n\n\n Â\xa0 Â\xa0 \n Â\xa0 Bibliographien\n\n Â\xa0 Â\xa0 \n60 Â\xa0 Bibliographien der Bibliographien\n\n Â\xa0 Â\xa0 \n65 Â\xa0 Bibliographien der Â»verkleideten LiteraturÂ« (Anonymen- und Pseudonymenlexika; falsches, fehlendes oder fingiertes Impressum)\n\n Â\xa0 Â\xa0 \n70 Â\xa0 internationale Allgemeinbibliographien\n\n Â\xa0 Â\xa0 \n75 Â\xa0 idem, nur Zeitschriften\n\n\n\n Â\xa0 Â\xa0 \n Â\xa0 Nationale Allgemeinbibliographien\n\n Â\xa0 Â\xa0 \n80 Â\xa0 italienische\n\n Â\xa0 Â\xa0 \n83 Â\xa0 idem, nur Zeitschriften\n\n Â\xa0 Â\xa0 \n85 Â\xa0 deutsche\n\n Â\xa0 Â\xa0 \n87 Â\xa0 idem, nur Zeitschriften\n\n Â\xa0 Â\xa0 \n89 Â\xa0 franzÃ¶sische\n\n Â\xa0 Â\xa0 \n91 Â\xa0 idem, nur Zeitschriften\n\n Â\xa0 Â\xa0 \n93 Â\xa0 englisch-amerikanische\n\n Â\xa0 Â\xa0 \n95 Â\xa0 idem, nur Zei